### Import required variables and paths

In [0]:
%run "./commons"

In [0]:
dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment or catalog in lower case')
env = dbutils.widgets.get("env")
print(env)

### Read Silver Traffic Table

In [0]:
def read_SilverTrafficTable(environment):
    print('Reading the Silver Traffic Table Data : ',end='')
    df_SilverTraffic = (spark.readStream.table(f"`{environment}`.`silver`.traffic"))
    print(f'Reading {environment}_catalog.silver.silver_traffic Success!')
    print("**********************************")
    return df_SilverTraffic

### Read Silver roads table 

In [0]:

def read_SilverRoadsTable(environment):
    print('Reading the Silver Table Silver_roads Data : ',end='')
    df_SilverRoads = (spark.readStream.table(f"`{environment}`.`silver`.roads"))
    print(f'Reading {environment}_catalog.silver.silver_roads Success!')
    print("**********************************")
    return df_SilverRoads

### Creating vehicle Intensity Column

In [0]:
def create_VehicleIntensity(df):
    from pyspark.sql.functions import col
    print('Creating Vehicle Intensity column : ',end='')
    df_veh = df.withColumn('Vehicle_Intensity',col('Motor_Vehicles_Count') / col('Link_length_km'))
    print("Success!!!")
    print('***************')
    return df_veh

### Creating LoadTime column

In [0]:
def create_LoadTime(df):
    from pyspark.sql.functions import current_timestamp
    print('Creating Load Time column : ',end='')
    df_timestamp = df.withColumn('Load_Time',
                      current_timestamp()
                      )
    print('Success!!')
    print('**************')
    return df_timestamp

### Writing Data to Gold Traffic

In [0]:
def write_traffic_gold(environment,df):
    write_gold_stream = (df.writeStream.format('delta')
        .option('checkpointLocation',f"{checkpoints}/GoldTrafficLoad/Checkpt/")
        .outputMode('append')
        .queryName("GoldTrafficWriteStream")
        .trigger(availableNow=True)
        .toTable(f"`{environment}`.`gold`.`gold_traffic`"))
    print('Writing to Gold table : ',end='')
    write_gold_stream.awaitTermination()

    print(f'Writing `{environment}`.`gold`.`gold_traffic` Success!')

### Write Data to Gold roads

In [0]:
def write_Roads_gold(environment,df):
    print('Writing the gold_roads Data : ',end='') 

    write_gold_roads = (df.writeStream
                .format('delta')
                .option('checkpointLocation',f"{checkpoints}/GoldRoadsLoad/Checkpt/")
                .outputMode('append')
                .queryName("GoldRoadsWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}`.`gold`.`gold_roads`"))
    
    write_gold_roads.awaitTermination()
    print(f'Writing `{environment}`.`gold`.`gold_roads` Success!')

### Calling all functions

In [0]:
# Reading silver tables
df_silver_traffic = read_SilverTrafficTable(environment=env)
df_silver_road = read_SilverRoadsTable(environment=env)

# Transformation
df_vehicle = create_VehicleIntensity(df_silver_traffic)
df_FinalTraffic = create_LoadTime(df_vehicle)
df_FinalRoads = create_LoadTime(df_silver_road)

# Write transformed data to gold tables
write_traffic_gold(env,df_FinalTraffic)
write_Roads_gold(env,df_FinalRoads)